# 数据类型

每个张量都有数据类型，在深度学习中通常是 `float32`，但也可以是 `int8`（例如，用于模型量化）和其他数据类型。在 {ref}`ch_vector_add_te` 中创建的 `tvm_vector_add`  模块只接受 `float32` 张量。在本节将其扩展到其他数据类型。

## 指定数据类型

要使用与默认值 `float32` 不同的数据类型，可以在创建占位符时显式指定。在下面的代码块中将定义在 {ref}`ch_vector_add_te` 中的泛型向量加法表达式，以接受参数 `dtype` 来指定数据类型。特别地，当创建 `A` 和 `B` 时，将 `dtype` 传递给 `te.placeholder` 占位符。结果 `C` 将获得与 `A` 和 `B` 相同的数据类型。

In [1]:
%cd ../..
import set_env

/media/pc/data/4tb/lxw/home/lxw/tvm-book/doc/tutorials


In [2]:
from tvm_book.contrib import d2ltvm

import tvm
from tvm import te
import numpy as np

n = 100

def tvm_vector_add(dtype):
    A = te.placeholder((n,), dtype=dtype)
    B = te.placeholder((n,), dtype=dtype)
    C = te.compute(A.shape, lambda i: A[i] + B[i])
    print('表达式 dtype:', A.dtype, B.dtype, C.dtype)
    s = te.create_schedule(C.op)
    m = tvm.lower(s, [A, B, C])
    return tvm.build(m)

编译接受 `int32` 张量的模块。

In [3]:
mod = tvm_vector_add('int32')

表达式 dtype: int32 int32 int32


然后，定义一个方法，用特定的数据类型验证结果。注意，传递了构造函数，通过 `astype` 修改张量数据类型。

In [4]:
def test_mod(mod, dtype):
    a, b, c = d2ltvm.get_abc(n, lambda x: tvm.nd.array(x.astype(dtype)))
    print('张量 dtype:', a.dtype, b.dtype, c.dtype)
    mod(a, b, c)
    np.testing.assert_equal(c.numpy(), a.numpy() + b.numpy())

test_mod(mod, 'int32')

张量 dtype: int32 int32 int32


也可以尝试其他数据类型：

In [5]:
for dtype in ['float16', 'float64', 'int8','int16', 'int64']:
    mod = tvm_vector_add(dtype)
    test_mod(mod, dtype)

表达式 dtype: float16 float16 float16
张量 dtype: float16 float16 float16
表达式 dtype: float64 float64 float64
张量 dtype: float64 float64 float64
表达式 dtype: int8 int8 int8
张量 dtype: int8 int8 int8
表达式 dtype: int16 int16 int16
张量 dtype: int16 int16 int16
表达式 dtype: int64 int64 int64
张量 dtype: int64 int64 int64


## 转换元素数据类型

除了构造具有特定数据类型的张量外，还可以在计算过程中强制转换张量元素的数据类型。下面的方法与 `tvm_vector_add` 相同，只是它在 `te.compute` 中强制转换了 `A` 和 `B` 的数据类型，而 `te. placeholder` 中定义的数据类型保留为默认值（`float32`）。由于 `astype` 执行了类型转换，结果 `C` 将具有 `dtype` 指定的数据类型。

In [6]:
def tvm_vector_add_2(dtype):
    A = te.placeholder((n,))
    B = te.placeholder((n,))
    C = te.compute(A.shape, 
                    lambda i: A[i].astype(dtype) + B[i].astype(dtype))
    print('表达式 dtype:', A.dtype, B.dtype, C.dtype)
    s = te.create_schedule(C.op)
    return tvm.build(s, [A, B, C])

然后定义相似的测试函数来验证结果。

In [7]:
def test_mod_2(mod, dtype):
    a, b, c = d2ltvm.get_abc(n)
    # by default `get_abc` returns NumPy ndarray in float32
    a_tvm, b_tvm = tvm.nd.array(a), tvm.nd.array(b)
    c_tvm = tvm.nd.array(c.astype(dtype))
    print('张量 dtype:', a_tvm.dtype, b_tvm.dtype, c_tvm.dtype)
    mod(a_tvm, b_tvm, c_tvm)
    np.testing.assert_equal(c_tvm.numpy(), a.astype(dtype) + b.astype(dtype))

mod = tvm_vector_add_2('int32')
test_mod_2(mod, 'int32')

表达式 dtype: float32 float32 int32
张量 dtype: float32 float32 int32


## 小结

- 在创建 TVM 占位符时，可以通过 `dtype` 指定数据类型。
- 在 TVM compute 中，张量元素的数据类型可以通过 `astype` 进行转换。